<a href="https://colab.research.google.com/github/Hoor01001000/HoorAK/blob/main/Enhancing_Data_Quality_in_Data_Warehouses_Using_AI_dashboard_tool_using_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok --quiet
from pyngrok import ngrok
import pandas as pd
import streamlit as st
from sklearn.ensemble import IsolationForest
from sklearn.impute import KNNImputer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [2]:
df = pd.read_csv("All Electronics.csv")

In [3]:
code = '''
import streamlit as st
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest

st.set_page_config(page_title="AI Data Quality Tool", layout="wide")
st.title("AI-Powered Data Quality Dashboard")

df = pd.read_csv("All Electronics.csv")

st.subheader("Original Dataset Preview")
st.dataframe(df.head())

df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
df['no_of_ratings'] = df['no_of_ratings'].astype(str).str.replace(",", "")
df['no_of_ratings'] = pd.to_numeric(df['no_of_ratings'], errors='coerce')
df['discount_price'] = df['discount_price'].astype(str).str.replace("₹", "").str.replace(",", "")
df['actual_price'] = df['actual_price'].astype(str).str.replace("₹", "").str.replace(",", "")
df['discount_price'] = pd.to_numeric(df['discount_price'], errors='coerce')
df['actual_price'] = pd.to_numeric(df['actual_price'], errors='coerce')

st.subheader(" Missing Value Overview")
missing_vals = df.isnull().sum()
st.write(missing_vals[missing_vals > 0])

st.subheader("KNN Imputation for Missing Values")
k = st.slider("Choose number of neighbors (K)", 1, 10, 5)
imputer = KNNImputer(n_neighbors=k)
numeric_cols = ['ratings', 'no_of_ratings', 'discount_price', 'actual_price']
df_imputed = df.copy()
df_imputed[numeric_cols] = imputer.fit_transform(df_imputed[numeric_cols])
st.success("Missing values filled using KNN")

st.subheader("Anomaly Detection (Isolation Forest)")
contamination = st.slider("Contamination Rate", 0.01, 0.1, 0.02)
iso = IsolationForest(contamination=contamination, random_state=42)
df_imputed['anomaly'] = iso.fit_predict(df_imputed[numeric_cols])
anomalies = df_imputed[df_imputed['anomaly'] == -1]
st.warning(f"Detected {len(anomalies)} anomalies")
st.dataframe(anomalies.head())

st.subheader("Cleaned Dataset")
df_cleaned = df_imputed.drop(columns='anomaly')
st.dataframe(df_cleaned.head())

csv = df_cleaned.to_csv(index=False).encode('utf-8')
st.download_button("Download Cleaned Dataset", csv, "cleaned_data.csv", "text/csv")
'''

with open("python.py", "w") as f:
    f.write(code)

In [4]:
!streamlit run python.py &> /dev/null &

In [5]:
from pyngrok import conf
conf.get_default().auth_token = "2vfT03bkXAM6hmeTmSNZeTWPoA6_49eMptgEKBnibB4jJWqUX"

In [6]:
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(addr="8501", proto="http")
print(" Streamlit ", public_url)

!streamlit run python.py &> /dev/null &

 Streamlit  NgrokTunnel: "https://0b85-34-168-104-139.ngrok-free.app" -> "http://localhost:8501"
